In [20]:
import streamlit as st
import pandas as pd
import altair as alt
import numpy as np
from utils import get_course_df
from utils import get_instructor_df
from utils import get_dept_df
from utils import options_map_to_columns

In [21]:
numeric_columns = [
        'hrs_per_week',
        'total_students',
        'responses',
        'response_rate',
        'interest_in_student_learning',
        'clearly_explain_course_requirements',
        'clear_learning_objectives_and_goals',
        'instructor_provides_feedback',
        'demonstrate_importance_of_subject_matter',
        'explains_subject_matter_of_course',
        'show_respect_for_all_students',
        'overall_teaching_rate',
        'overall_course_rate'
    ]

In [22]:
def load_data():
    # Load the yelp data.
    fce_data_url = "../fce-data.csv"
    raw_df = pd.read_csv(fce_data_url)
    df = raw_df
    df.dropna(inplace=True)
    # Convert numeric columns
    for nc in numeric_columns:
        df[nc] = pd.to_numeric(df[nc])

    # Filter data
    df = df[(df['total_students'] > 5) & (df['total_students'] <= 325)] # Remove Outlier
    df = df[df['response_rate'] > 0.4]
    df = df[df['college'] != 'Teaching Assistants']

    # df['overall_rate'] = df[(df['overall_course_rate'] + df['overall_teaching_rate'] + df['show_respect_for_all_students']) / 3]
    return df

In [23]:
df = load_data()
course_df = get_course_df(df)

options_map_to_column = options_map_to_columns

In [53]:
options_columns = ['overall_teaching_rate', 'overall_course_rate']
course_options = ['2021 Fall 85738 ED GOALS INST ASSESS', '2021 Spring 5840 TOOLS ONLINE LRNG', '2021 Fall 10301 MACHINE LEARNING']
course_df["course_name_code_year"] = course_df["year"].astype(str) + ' ' + course_df["semester"] + ' '  + course_df["num"].astype(str) + ' ' + course_df["course_name"]
course_df["course_name_code"] = course_df["num"].astype(str) + ' ' + course_df["course_name"]


In [58]:
# 2021 Fall 10301 MACHINE LEARNING

course_df2 = course_df.groupby(['course_name_code', 'year']).agg(
                course_name_code_year = ('course_name_code_year', 'first'),
                hrs_per_week = ('hrs_per_week', 'mean'),
                course_name=('course_name', 'max'), 
                responses=('responses', 'mean'), 
                num=('num', 'max'), 
                instructor_combined=('instructor', lambda x: ' - '.join(x)), 
                interest_in_student_learning=('interest_in_student_learning', 'mean'),
                clearly_explain_course_requirements=('clearly_explain_course_requirements', 'mean'),
                clear_learning_objectives_and_goals=('clear_learning_objectives_and_goals', 'mean'),
                instructor_provides_feedback=('instructor_provides_feedback', 'mean'),
                demonstrate_importance_of_subject_matter=('demonstrate_importance_of_subject_matter', 'mean'),
                explains_subject_matter_of_course=('explains_subject_matter_of_course', 'mean'),
                show_respect_for_all_students=('show_respect_for_all_students', 'mean'),
                overall_teaching_rate=('overall_teaching_rate', 'mean'),
                overall_course_rate=('overall_course_rate', 'mean'))
                
# course_df2.loc[course_df2['course_name_code_year'] == '2021 Fall 10301 MACHINE LEARNING']

In [64]:
filter_df1 = course_df2.loc[course_df2['course_name_code_year'].isin(course_options)]
columns_filter = options_columns + ['course_name_code_year', 'instructor_combined', 'hrs_per_week', 'responses']

filter_df1 = filter_df1[columns_filter]
reshaped_filter_df = filter_df1.melt(id_vars=['course_name_code_year', 'instructor_combined', 'hrs_per_week', 'responses'], var_name = 'Judge Parameter', value_name = 'Rating').sort_values(by='course_name_code_year').reset_index(drop=True)
reshaped_filter_df

,course_name_code_year,instructor_combined,hrs_per_week,responses,Judge Parameter,Rating
0,2021 Fall 10301 MACHINE LEARNING,"GORMLEY, MATTHEW - MITCHELL, TOM",13.7125,39.0,overall_teaching_rate,4.5850
1,2021 Fall 10301 MACHINE LEARNING,"GORMLEY, MATTHEW - MITCHELL, TOM",13.7125,39.0,overall_course_rate,4.4675
2,2021 Fall 85738 ED GOALS INST ASSESS,"HERCKIS, LAUREN",19.0000,27.0,overall_teaching_rate,4.5600
3,2021 Fall 85738 ED GOALS INST ASSESS,"HERCKIS, LAUREN",19.0000,27.0,overall_course_rate,4.2200
4,2021 Spring 5840 TOOLS ONLINE LRNG,"KULKARNI, CHINMAY",8.0000,7.0,overall_teaching_rate,4.0000
5,2021 Spring 5840 TOOLS ONLINE LRNG,"KULKARNI, CHINMAY",8.0000,7.0,overall_course_rate,4.0000


In [28]:
filter_df1 = course_df.groupby(['course_name_code_year', 'instructor'])
# [options_columns].mean().reset_index()
filter_df1.count()


,,year,semester,num,course_name,college,dept,section,course_level,hrs_per_week,total_students,...,response_rate,interest_in_student_learning,clearly_explain_course_requirements,clear_learning_objectives_and_goals,instructor_provides_feedback,demonstrate_importance_of_subject_matter,explains_subject_matter_of_course,show_respect_for_all_students,overall_teaching_rate,overall_course_rate
course_name_code_year,instructor,,,,,,,,,,,,,,,,,,,,,
2011 Spring 10601 MACHINE LEARNING,"ROSENFELD, RONALD",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2011 Spring 10701 MACHINE LEARNING,"MITCHELL, TOM",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2011 Spring 10702 STATISTCL MACH LRNG,"WASSERMAN, LARRY",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2011 Spring 10831 SP TPC: MCH LN & PLY,"NEILL, DANIEL",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2011 Spring 11344 MACHINE LRNG PRACTCE,"ROSE, CAROLYN",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022 Spring 99357 IDEATE: PHOTOGRAPHY,"DYAR, HUGH",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2022 Spring 99358 IDEATE UNITY3D INTRO,"DYAR, HUGH",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2022 Spring 99359 IDEATE: 3D MOD/PRINT,"PATTON, JAYLA",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [18]:
filter_df1 = course_df.groupby(['course_name_code_year', 'instructor', 'responses'])[options_columns].mean().reset_index()
# st.write(filter_df1)
filter_df = filter_df1.loc[course_df['course_name_code_year'].isin(course_options)]
# st.write(filter_df)
reshaped_filter_df = filter_df.melt(id_vars=['course_name_code_year', 'instructor', 'responses'], var_name = 'Judge Parameter', value_name = 'Rating').sort_values(by='course_name_code_year').reset_index(drop=True)
# st.write(reshaped_filter_df)
selection_legend = alt.selection_multi(fields=['course_name_code_year'], bind='legend')

base = alt.Chart(reshaped_filter_df).mark_bar(tooltip=True).transform_calculate(
    y="split(datum.y, '_')"
).encode(
    y=alt.Y('course_name_code_year:O', type="nominal", axis=alt.Axis(title=None, labels=False)),
    x=alt.X('Rating:Q',axis=alt.Axis(grid=False)),
    color=alt.Color('course_name_code_year:N',legend=alt.Legend(title="Course Name", labelFontSize=12)),
    opacity=alt.condition(selection_legend, alt.value(1), alt.value(0.2)),
    row=alt.Row('Judge Parameter:N', header=alt.Header(labelAngle=0, labelAlign="left", labelFontSize=8)),
    tooltip = [alt.Tooltip(field = "course_name_code_year", title = "Course Name", type = "nominal"),
            alt.Tooltip(field = "Rating", title ="Rating", type = "quantitative", format=".2f"),
            alt.Tooltip(field = "instructor", title ="Instructor", type = "nominal"),
            alt.Tooltip(field = "responses", title ="Responses", type = "quantitative")
    ],
).properties(
    title="Compare Ratings for Selected Courses", 
    width=400,
).configure_title(
    anchor='middle',
).add_selection(
    selection_legend
).configure_legend(
    labelLimit= 0
)


In [19]:
base

alt.Chart(...)

In [77]:
dept_df = get_dept_df(df)
dept_df["dept_name_college"] = dept_df["dept"] + ' '  + dept_df["college"]


In [108]:
dept_df2 = dept_df.groupby(['dept_name_college', 'year']).agg(
    dept_name_college = ('dept_name_college', 'first'),
    total_students=('total_students', 'sum'),
    hrs_per_week = ('hrs_per_week', 'mean'),
    responses=('responses', 'sum'), 
    interest_in_student_learning=('interest_in_student_learning', 'mean'),
    clearly_explain_course_requirements=('clearly_explain_course_requirements', 'mean'),
    clear_learning_objectives_and_goals=('clear_learning_objectives_and_goals', 'mean'),
    instructor_provides_feedback=('instructor_provides_feedback', 'mean'),
    demonstrate_importance_of_subject_matter=('demonstrate_importance_of_subject_matter', 'mean'),
    explains_subject_matter_of_course=('explains_subject_matter_of_course', 'mean'),
    show_respect_for_all_students=('show_respect_for_all_students', 'mean'),
    overall_teaching_rate=('overall_teaching_rate', 'mean'),
    overall_course_rate=('overall_course_rate', 'mean'))

In [109]:
dept_df2

dept_name_college  \
dept_name_college                   year                                        
AEM Heinz College                   2019                    AEM Heinz College   
                                    2020                    AEM Heinz College   
                                    2021                    AEM Heinz College   
                                    2022                    AEM Heinz College   
ARC College of Fine Arts            2011             ARC College of Fine Arts   
...                                                                       ...   
SV Carnegie Institute of Technology 2011  SV Carnegie Institute of Technology   
                                    2012  SV Carnegie Institute of Technology   
                                    2013  SV Carnegie Institute of Technology   
                                    2014  SV Carnegie Institute of Technology   
                                    2015  SV Carnegie Institute of Technology   

                                          total_students  hrs_per_week  \
dept_name_college                   year                                 
AEM Heinz College                   2019       20.857143      6.232857   
                                    2020       47.600000      8.037778   
                                    2021       39.133333      7.490833   
                                    2022       17.500000      5.500000   
ARC College of Fine Arts            2011       29.813953     10.345116   
...                                                  ...           ...   
SV Carnegie Institute of Technology 2011       40.333333     10.582500   
                                    2012       41.687500      8.750000   
                                    2013       55.133333     13.353407   
                                    2014       53.650980     12.571520   
                                    2015       39.230769     14.176462   

                                          responses  \
dept_name_college                   year              
AEM Heinz College                   2019  16.285714   
                                    2020  35.422222   
                                    2021  24.200000   
                                    2022  11.500000   
ARC College of Fine Arts            2011  16.418605   
...                                             ...   
SV Carnegie Institute of Technology 2011  19.500000   
                                    2012  26.500000   
                                    2013  35.577778   
                                    2014  38.756583   
                                    2015  27.523077   

                                          interest_in_student_learning  \
dept_name_college                   year                                 
AEM Heinz College                   2019                      4.601429   
                                    2020                      4.550556   
                                    2021                      4.657500   
                                    2022                      4.060000   
ARC College of Fine Arts            2011                      4.039535   
...                                                                ...   
SV Carnegie Institute of Technology 2011                      4.345000   
                                    2012                      4.490417   
                                    2013                      4.541259   
                                    2014                      4.556963   
                                    2015                      4.433308   

                                          clearly_explain_course_requirements  \
dept_name_college                   year                                        
AEM Heinz College                   2019                             4.254286   
                                    2020                             4.205889   
                                    2021

In [113]:
# dept_df["dept_name_college"] = dept_df["dept"] + ' '  + dept_df["college"]
options_dept=['ML Dietrich College of Humanities and Social Sciences', 'CS School of Computer Science', 'BMD Carnegie Institute of Technology']


In [118]:
# filter_df1 = dept_df.loc[dept_df['year'] == 2021]
filter_df1 = dept_df2.loc[dept_df2['dept_name_college'].isin(options_dept)]

columns_filter = options_columns + ['dept_name_college', 'total_students', 'hrs_per_week', 'responses']


filter_df3 = filter_df1[columns_filter]

In [115]:
filter_df3

overall_teaching_rate  \
dept_name_college                                  year                          
BMD Carnegie Institute of Technology               2011               4.273529   
                                                   2012               4.168409   
                                                   2013               4.331474   
                                                   2014               4.225168   
                                                   2015               4.265273   
                                                   2016               4.391023   
                                                   2017               4.396176   
                                                   2018               4.484375   
                                                   2019               4.294876   
                                                   2020               4.545000   
                                                   2021               4.483339   
CS School of Computer Science                      2011               4.264884   
                                                   2012               4.156232   
                                                   2013               4.180257   
                                                   2014               4.283592   
                                                   2015               4.332315   
                                                   2016               4.368252   
                                                   2017               4.375557   
                                                   2018               4.319735   
                                                   2019               4.387425   
                                                   2020               4.443763   
                                                   2021               4.397252   
ML Dietrich College of Humanities and Social Sc... 2011               4.564110   
                                                   2012               4.474747   
                                                   2013               4.592601   
                                                   2014               4.704488   
                                                   2015               4.721341   
                                                   2016               4.768974   
                                                   2017               4.641068   
                                                   2018               4.771067   
                                                   2019               4.577183   
                                                   2020               4.766019   
                                                   2021               4.783132   
                                                   2022               5.000000   

                                                         overall_course_rate  \
dept_name_college                                  year                        
BMD Carnegie Institute of Technology               2011             4.228824   
                                                   2012             4.029631   
                                                   2013             4.299957   
                                                   2014             4.148754   
                                                   2015             4.200909   
                                                   2016             4.335750   
                                                   2017             4.335253   
                                                   2018             4.363889   
                                                   2019             4.177215   
                                                   2020             4.484000   
                                                   2021             4.457500   
CS School of Computer Science                      2011         